<h1 style="text-align:center">Segmenting and Clustering Neighbourhood in Toronto</h1>

In [54]:
import numpy as np
import pandas as pd
import json
import requests
from geopy.geocoders import Nominatim
from pandas.io.json import json_normalize
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
import folium
from bs4 import BeautifulSoup
print("Library successfully imported")

Library successfully imported


In [55]:
url='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
results=requests.get(url).text
soup = BeautifulSoup(results)


In [56]:
table_contents=[]
table=soup.find('table')
for row in table.findAll('td'):
    cell = {}
    if row.span.text=='Not assigned':
        pass
    else:
        cell['PostalCode'] = row.p.text[:3]
        cell['Borough'] = (row.span.text).split('(')[0]
        cell['Neighborhood'] = (((((row.span.text).split('(')[1]).strip(')')).replace(' /',',')).replace(')',' ')).strip(' ')
        table_contents.append(cell)

# print(table_contents)
df=pd.DataFrame(table_contents)
df['Borough']=df['Borough'].replace({'Downtown TorontoStn A PO Boxes25 The Esplanade':'Downtown Toronto Stn A',
                                             'East TorontoBusiness reply mail Processing Centre969 Eastern':'East Toronto Business',
                                             'EtobicokeNorthwest':'Etobicoke Northwest','East YorkEast Toronto':'East York/East Toronto',
                                             'MississaugaCanada Post Gateway Processing Centre':'Mississauga'})

In [57]:
df=df.sort_values('PostalCode')
df.shape

(103, 3)

In [58]:
url='https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DS0701EN-SkillsNetwork/labs_v1/Geospatial_Coordinates.csv'
gta_longlat=pd.read_csv(url)
gta_longlat=gta_longlat.sort_values('Postal Code')
gta_longlat.shape

(103, 3)

In [59]:
gta_df = df
gta_df['Latitude'] = gta_longlat['Latitude']
gta_df['Longitude']= gta_longlat['Longitude']
gta_df

,PostalCode,Borough,Neighborhood,Latitude,Longitude
6,M1B,Scarborough,"Malvern, Rouge",43.727929,-79.262029
12,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.794200,-79.262029
18,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.778517,-79.346556
22,M1G,Scarborough,Woburn,43.770120,-79.408493
26,M1H,Scarborough,Cedarbrae,43.745906,-79.352188
...,...,...,...,...,...
64,M9N,York,Weston,43.696948,-79.411307
70,M9P,Etobicoke,Westmount,43.648429,-79.382280
77,M9R,Etobicoke,"Kingsview Village, St. Phillips, Martin Grove ...",43.647927,-79.419750
89,M9V,Etobicoke,"South Steeles, Silverstone, Humbergate, Jamest...",43.602414,-79.543484


In [60]:
address = 'Toronto, CA'

geolocator = Nominatim(user_agent="gta_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


In [61]:
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(gta_df['Latitude'], gta_df['Longitude'], gta_df['Borough'], gta_df['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto